# Reporte de espacio IPv4 no anunciado

Este reporte incluye información acerca de espacio IPv4 asignad por LACNIC pero no visible en la tabla de enrutamiento global.

## Metodología

Se toman las siguientes fuentes de datos:

- "delegated-extended" de LACNIC
- RIPE RIS IPv4 Dump

El algoritmo de alto nivel es:

1. Recorrer el delegated extended
   1. Para cada organización, tomar todas sus asignaciones
   2. Producir un agregado de las mismas
2. Recorrer el producto de (1)
   1. Para cada ruta de RIS, identificar el agregado de donde proviene, y de allí la organización a la que pertenece
3. Recorrer el producto de (2)
   1. Para cada agregado, agregar todas las rutas que le corresponden y de allí calcular cuanto del total todas esas rutas cubren.
   
## Producto del algoritmo

El producto es un archivo separado por pipes con la siguiente estructura:

```
orgid|prefix|visible|dark|total
20118|24.232.0.0/16|65536|0|65536
272078|45.4.0.0/22|1024|0|1024
271038|45.4.4.0/22|1024|0|1024
261330|45.4.8.0/22|1024|0|1024

...
...
258668|45.4.60.0/22|1024|0|1024
258825|45.4.64.0/22|1024|0|1024
```

Las columnas:

- orgid: coincide con el opaque-id del delegated-extended, identifica la organización
- prefijo: prefijo IPv4 
- visible: cantidad de direcciones de este prefijo visibles (es decir, contenidas en rutas)
- dark: cantidad de direcciones no-visibles
- total: cantidad total de direcciones contenidas en el prefijo



## Resultados

### R1: Cantidad de asignaciones completamente invisibles, por organización

En este resultado se listan las asignaciones completamente invisibles (es decir, sin ninguna ruta que las cubra) y las organizaciones que las tienen.

In [57]:
import pandas
import sqlite3

w_date = '20191110'
conn = sqlite3.connect(":memory:")

inf = "s1_invisible_prefixes-{}.csv".format(w_date)
df = pandas.read_csv(inf, delimiter='|')
df.to_sql("invr", conn, if_exists='append', index=False)

df.head()

,orgid,prefix,visible,dark,total
0,20118,24.232.0.0/16,65536,0,65536
1,272078,45.4.0.0/22,1024,0,1024
2,271038,45.4.4.0/22,1024,0,1024
3,261330,45.4.8.0/22,1024,0,1024
4,265673,45.4.12.0/22,1024,0,1024


### R2. Cantidad de asignaciones invisibles mayores o iguales a un /16

In [58]:
nva = df.query("visible == 0 and total > 16384").sort_values("total", ascending=False)
# nva.head(20)
print("Cantidad de asignaciones grandes no visibles: {}".format(len(nva['total'])))

Cantidad de asignaciones grandes no visibles: 26


In [59]:
print("Valor aproximado de mercado de las direcciones no visibles en bloques grandes: {} usd".format(nva['total'].sum(axis=0)*15))

Valor aproximado de mercado de las direcciones no visibles en bloques grandes: 28508160 usd


### R2.1: Listado de asignaciones grandes no visibles

In [61]:
nva.to_csv("s1_bigallocs_notvisible-{}.csv".format(w_date, delimiter='|'))

nva

,orgid,prefix,visible,dark,total
9914,47779,186.172.0.0/14,0,262144,262144
9911,23085,186.164.0.0/15,0,131072,131072
9877,35073,186.98.0.0/15,0,131072,131072
3537,95942,129.90.0.0/16,0,65536,65536
5569,12155,163.250.0.0/16,0,65536,65536
9902,38660,186.157.0.0/16,0,65536,65536
5656,95936,167.134.0.0/16,0,65536,65536
5652,245143,167.28.0.0/16,0,65536,65536
5649,13630,166.110.0.0/16,0,65536,65536
5032,65334,140.191.0.0/16,0,65536,65536


### R3. Porcentaje de espacio no visible respecto del espacio asignado

In [ ]:
dark_t = df['dark'].sum()
visible_t = df['visible'].sum()
total_t = df['total'].sum()

print("El porcentaje de espacio asignado pero no visible: {0:.2f}%".format(dark_t / total_t * 100))

### R4. Cantidad de asignaciones con el mayor espacio oscuro, pero parcialmente visibles

In [94]:
nva_parcial = df.query("dark>0").sort_values('dark', ascending=False)
# nva_parcial_agg = nva_parcial.groupby('orgid')
nva_parcial.sort_values('dark', ascending=False).head(10)

,orgid,prefix,visible,dark,total
11921,21461,201.96.0.0/11,1464064,633088,2097152
10541,21461,189.128.0.0/10,3690496,503808,4194304
10384,21461,187.192.0.0/11,1617408,479744,2097152
10369,21461,187.128.0.0/11,1703936,393216,2097152
11018,35073,191.104.0.0/13,131072,393216,524288
10385,21461,187.224.0.0/12,673792,374784,1048576
9621,20118,181.96.0.0/12,716800,331776,1048576
10561,21461,189.224.0.0/11,1819136,278016,2097152
9771,236102,181.216.0.0/13,262144,262144,524288
9914,47779,186.172.0.0/14,0,262144,262144


In [95]:
nva_parcial.groupby('orgid').sum().sort_values('dark', ascending=False).head(10)

,visible,dark,total
orgid,,,
21461,12292864,3056384,15349248
23085,546816,1312768,1859584
35073,1666048,1119232,2785280
20118,4053504,943616,4997120
236102,5158656,706816,5865472
21455,1162752,492032,1654784
47779,232448,431104,663552
22675,285184,416256,701440
53158,643328,302848,946176


In [96]:
nva_parcial_agg.get_group(35073).sum()

orgid                                                 561168
prefix     191.104.0.0/13186.98.0.0/15186.168.0.0/14152.2...
visible                                              1666048
dark                                                 1119232
total                                                2785280
dtype: object